
https://tianchi.aliyun.com/competition/entrance/231593/introduction
官方網站

https://redstonewill.com/1681/
紅色石頭: 介紹與範例

https://github.com/RedstoneWill/MachineLearningInAction-Camp/blob/master/Week1/o2o-1.ipynb
(0.532344469452)

https://github.com/RedstoneWill/MachineLearningInAction-Camp/blob/master/Week5/Assignments/o2o-1.ipynb
(0.532344469452)

https://github.com/RedstoneWill/MachineLearningInAction-Camp/blob/master/Week10/Assignments/o2o-2.ipynb
(0.630400363653)

https://github.com/wepe/O2O-Coupon-Usage-Forecast
第一名隊伍的註解

https://www.kaggle.com/lianglirong/alibaba-tianchi-o2o
(0.9094526220772331)
範例

https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.15.29281b48IrTtER&postId=8462
(0.909452622077)
100行代码入门天池O2O优惠券使用新人赛【精简教程版】

https://www.kaggle.com/cbrogan/xgboost-example-python
XGBoost example (Python)


In [1]:
import os
import numpy as np
import pandas as pd
from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

DATA_ROOT = "./data/ml100marathon-02-01/"

In [2]:
dfoff = pd.read_csv(os.path.join(DATA_ROOT,'train_offline.csv'))
dftest = pd.read_csv(os.path.join(DATA_ROOT,'test_offline.csv'))
dftest = dftest[~dftest.Coupon_id.isna()]
dftest.reset_index(drop=True, inplace=True)
print('train data: ', dfoff.shape)
print('test data: ', dftest.shape)

train data:  (1160742, 7)
test data:  (306313, 6)


In [3]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN


In [4]:
dftest.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,1439408,4663,11002.0,150:20,1.0,20160528.0
1,1439408,2632,8591.0,20:1,0.0,20160613.0
2,1439408,2632,8591.0,20:1,0.0,20160516.0
3,2029232,450,1532.0,30:5,0.0,20160530.0
4,2029232,6459,12737.0,20:1,0.0,20160519.0


In [5]:
#資料量太大了，所以開發時先用較少量的資料來玩，調整差不多後，再用完整資料來玩
#dfoff = dfoff[:100000]
#dftest = dftest[:10000]

print(dfoff.shape)
print(dftest.shape)

(1160742, 7)
(306313, 6)


In [6]:
## Creat target label 
"""
According to the definition, 
1) buy with coupon within (include) 15 days ==> 1
2) buy with coupon but out of 15 days ==> 0
3) buy without coupon ==> -1 (we don't care)
"""
def label(row):
    if np.isnan(row['Date_received']):
        return -1
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

dfoff["label"] = dfoff.apply(label, axis=1)
dfoff["label"].value_counts()

 0    710665
-1    413773
 1     36304
Name: label, dtype: int64

In [7]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0


In [8]:
# Generate features - weekday acquired coupon
def getWeekday(row):
    if (np.isnan(row)) or (row==-1):
        return row
    else:
        return pd.to_datetime(row, format = "%Y%m%d").dayofweek+1 # add one to make it from 0~6 -> 1~7

dfoff['weekday'] = dfoff['Date_received'].apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].apply(getWeekday)

# weekday_type (weekend = 1)
dfoff['weekday_type'] = dfoff['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to trainset
dftest['weekday_type'] = dftest['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to testset

In [9]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0


In [10]:
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
print(weekdaycols)

tmpdf = pd.get_dummies(dfoff['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [11]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0,0,0,0,0,0,0,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,0,0,1,0,0,0,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0,0,0,0,0,0,1,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0,0,0,0,0,1,0,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0,0,0,0,0,1,0,0


In [12]:
# Generate features - coupon discount and distance
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

def convertRate(row):
    """Convert discount to rate"""
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

def processData(df):
    
    # convert discunt_rate
    df['discount_rate'] = df['Discount_rate'].astype('str').apply(convertRate)
    df['discount_man'] = df['Discount_rate'].astype('str').apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].astype('str').apply(getDiscountJian)
    df['discount_type'] = df['Discount_rate'].astype('str').apply(getDiscountType)
    
    # convert distance
    df.loc[df.Distance.isna(), "Distance"] = 99
    return df

dfoff = processData(dfoff)
dftest = processData(dftest)

In [13]:
print('折扣種類: ', dfoff['discount_rate'].unique())

折扣種類:  [       nan 0.95       0.9        0.5        0.85       0.75
 0.83333333 0.8        0.93333333 0.7        0.6        0.86666667
 0.66666667 0.96666667 0.98       0.99       0.975      0.33333333
 0.2        0.4       ]


In [14]:
print('距離種類: ', dfoff['Distance'].unique())

距離種類:  [ 0.  1.  2. 99. 10.  4.  7.  9.  3.  5.  6.  8.]


In [15]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,...,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,discount_rate,discount_man,discount_jian,discount_type
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1,NaN,0,...,0,0,0,0,0,0,NaN,0,0,0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,...,0,1,0,0,0,0,0.95,20,1,1
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0,...,0,0,0,0,1,0,0.95,20,1,1
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0,...,0,0,0,1,0,0,0.90,200,20,1
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0,...,0,0,0,1,0,0,0.90,200,20,1


# 訓練用的 feature

In [16]:
original_feature = ['discount_rate',
                    'discount_type',
                    'discount_man', 
                    'discount_jian',
                    'Distance', 
                    'weekday', 
                    'weekday_type'] + weekdaycols
print(len(original_feature),original_feature)

predictors = original_feature
print(predictors)

14 ['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']
['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


# 切 訓練&驗證 用的資料

In [17]:
## Naive model
def split_train_valid(row, date_cut="20160416"):
    is_train = True if pd.to_datetime(row, format="%Y%m%d") < pd.to_datetime(date_cut, format="%Y%m%d") else False
    return is_train
    
df = dfoff[dfoff['label'] != -1].copy()
df["is_train"] = df["Date_received"].apply(split_train_valid)
train = df[df["is_train"]]
valid = df[~df["is_train"]]
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

print("Train size: {}, #positive: {}".format(len(train), train["label"].sum()))
print("Valid size: {}, #positive: {}".format(len(valid), valid["label"].sum()))

Train size: 667753, #positive: 32472
Valid size: 79216, #positive: 3832


# 建模型

In [18]:
def check_model(data, predictors):
    
    classifier = lambda: SGDClassifier(
        loss='log', 
        penalty='elasticnet', 
        fit_intercept=True, 
        max_iter=100, 
        shuffle=True, 
        n_jobs=1,
        class_weight=None)

    model = Pipeline(steps=[
        ('ss', StandardScaler()),
        ('en', classifier())
    ])

    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }

    folder = StratifiedKFold(n_splits=3, shuffle=True)
    
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1, 
        verbose=1)
    grid_search = grid_search.fit(data[predictors], 
                                  data['label'])
    
    return grid_search

model = check_model(train, predictors)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   18.1s finished


# 驗證預測

In [19]:
y_valid_pred = model.predict_proba(valid[predictors])
valid1 = valid.copy()
valid1['pred_prob'] = y_valid_pred[:, 1]

In [20]:
valid1.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,...,weekday_4,weekday_5,weekday_6,weekday_7,discount_rate,discount_man,discount_jian,discount_type,is_train,pred_prob
0,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0,...,0,1,0,0,0.900000,200,20,1,False,0.006850
1,163606,1569,5054.0,200:30,10.0,20160421.0,NaN,0,4.0,0,...,1,0,0,0,0.850000,200,30,1,False,0.006091
2,4061024,3381,7610.0,200:20,10.0,20160426.0,NaN,0,2.0,0,...,0,0,0,0,0.900000,200,20,1,False,0.009278
3,106443,450,3732.0,30:5,99.0,20160429.0,NaN,0,5.0,0,...,0,1,0,0,0.833333,30,5,1,False,0.059101
4,114747,1569,5054.0,200:30,9.0,20160426.0,NaN,0,2.0,0,...,0,0,0,0,0.850000,200,30,1,False,0.006269


# 評分

In [21]:
from sklearn.metrics import roc_auc_score, accuracy_score

auc_score = roc_auc_score(y_true=valid.label, y_score=y_valid_pred[:,1])
acc = accuracy_score(y_true=valid.label, y_pred=y_valid_pred.argmax(axis=1))

print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.742, Accuracy: 0.952


# 輸出預測檔案

In [22]:
def exportSubmit(model, filename):

    targetset = dftest.copy()
    print(targetset.shape)

    targetset = targetset[~targetset.Coupon_id.isna()]
    targetset.reset_index(drop=True, inplace=True)
    testset = targetset[predictors].copy()

    y_test_pred = model.predict_proba(testset[predictors])
    test1 = testset.copy()
    test1['pred_prob'] = y_test_pred[:, 1]
    print(test1.shape)
    
    output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
    print(output.shape)

    output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
    output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
    output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
    output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
    output.reset_index(drop=True, inplace=True)
    
    ### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
    out = output.groupby("uid", as_index=False).mean()
    out = out[["uid", "pred_prob"]]
    out.columns = ["uid", "label"]
    out.to_csv(filename, header=["uid", "label"], index=False) # submission format
    out.head()

In [23]:
exportSubmit(model, "Day_051_HW_baseline.csv")

(306313, 19)
(306313, 15)
(306313, 4)


# 使用 LogisticRegression

In [24]:
# 使用三種模型 : 邏輯斯迴歸 / 梯度提升機 / 隨機森林, 參數使用 Random Search 尋找
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

In [25]:
# 線性迴歸預測檔 (結果有部分隨機, 請以 Kaggle 計算的得分為準, 以下模型同理)
lr = LogisticRegression(tol=0.001, penalty='l2', fit_intercept=True, C=1.0)

lr.fit(train[predictors], train['label'])
lr_pred = lr.predict_proba(valid[predictors])

auc_score = roc_auc_score(y_true=valid.label, y_score=lr_pred[:,1])
acc = accuracy_score(y_true=valid.label, y_pred=lr_pred.argmax(axis=1))

print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

exportSubmit(lr, "Day_051_HW_lr.csv")
#0.58872

C:\Users\Ben\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Validation AUC: 0.747, Accuracy: 0.952
(306313, 19)
(306313, 15)
(306313, 4)


# 使用 GradientBoostingClassifier

In [26]:
# 梯度提升機預測檔 
gdbt = GradientBoostingClassifier(tol=100, subsample=0.75, n_estimators=250, max_features=len(predictors),
                                  max_depth=6, learning_rate=0.03)

gdbt.fit(train[predictors], train['label'])
gdbt_pred = gdbt.predict_proba(valid[predictors])

auc_score = roc_auc_score(y_true=valid.label, y_score=gdbt_pred[:,1])
acc = accuracy_score(y_true=valid.label, y_pred=gdbt_pred.argmax(axis=1))

print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

exportSubmit(gdbt, "Day_051_HW_gdbt.csv")
#0.62350

Validation AUC: 0.788, Accuracy: 0.952
(306313, 19)
(306313, 15)
(306313, 4)


# 使用 RandomForestClassifier

In [27]:
# 隨機森林預測檔
rf = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=1, 
                            max_features='sqrt', max_depth=6, bootstrap=True)

rf.fit(train[predictors], train['label'])
rf_pred = rf.predict_proba(valid[predictors])

auc_score = roc_auc_score(y_true=valid.label, y_score=rf_pred[:,1])
acc = accuracy_score(y_true=valid.label, y_pred=rf_pred.argmax(axis=1))

print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

exportSubmit(rf, "Day_051_HW_rf.csv")
#0.63762

Validation AUC: 0.791, Accuracy: 0.952
(306313, 19)
(306313, 15)
(306313, 4)


# 使用 xgboost

In [32]:
import xgboost as xgb

In [33]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train[predictors], train['label'])
gbm_pred = gbm.predict(valid[predictors])

In [37]:
auc_score = roc_auc_score(y_true=valid.label, y_score=gbm_pred)
acc = accuracy_score(y_true=valid.label, y_pred=gbm_pred)

print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.500, Accuracy: 0.952


In [38]:
exportSubmit(gbm, "Day_051_HW_xgboost.csv")
#0.63762

(306313, 19)
(306313, 15)
(306313, 4)
